# Analyzing time-dependent data

In [25]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import torch
import timeit

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from datetime import datetime
from umap import UMAP

from tqdm.notebook import tqdm

from src.LSTM import LSTMAutoencoder

In [2]:
FIGURES_DIR = './figures/'
MODELS_DIR = './src/models'
DATA_DIR = './csv/Pain_Plot_Features'
DATASETS = ['A', 'B', 'C', 'D', 'E']
GROUPS = ['pre', 'post']
DIRECTION = ['left', 'right']

# Load the data
data = {}
directory = os.listdir(DATA_DIR)
for file in directory:
    if file.endswith('.csv'):
        components = file.split('_')

        ## Assuming the file naming convention is:
        # dataset_group_mouse_direction_run.csv
        dataset = components[0]
        group = components[1]
        mouse = components[2]
        direction = components[3]
        run = components[4]

        datagroup = dataset + '_' + group
        if datagroup not in data:
            data[datagroup] = {}

        mouse_direction = mouse + '_' + direction
        if mouse_direction not in data[datagroup]:
            data[datagroup][mouse_direction] = {}

        data[datagroup][mouse_direction][run] = pd.read_csv(
            os.path.join(DATA_DIR, file), index_col=0)

In [3]:
## Print the data structure
for datagroup, mice in data.items():
    print(f"Group: {datagroup}")
    for mouse_direction, runs in mice.items():
        print(f"\t{mouse_direction}: {len(runs)} runs with shapes: {[df.shape for df in runs.values()]}")

Group: A_postDLC
	mouse10_left: 4 runs with shapes: [(418, 199), (303, 199), (331, 199), (237, 199)]
	mouse10_right: 3 runs with shapes: [(376, 199), (278, 199), (348, 199)]
	mouse1_left: 3 runs with shapes: [(310, 199), (313, 199), (297, 199)]
	mouse1_right: 4 runs with shapes: [(259, 199), (251, 199), (233, 199), (351, 199)]
	mouse2_left: 4 runs with shapes: [(209, 199), (216, 199), (124, 199), (155, 199)]
	mouse2_right: 3 runs with shapes: [(223, 199), (193, 199), (211, 199)]
	mouse3_left: 5 runs with shapes: [(249, 199), (231, 199), (241, 199), (318, 199), (244, 199)]
	mouse3_right: 4 runs with shapes: [(297, 199), (230, 199), (154, 199), (224, 199)]
	mouse4_left: 3 runs with shapes: [(269, 199), (188, 199), (221, 199)]
	mouse4_right: 3 runs with shapes: [(283, 199), (213, 199), (195, 199)]
	mouse5_left: 3 runs with shapes: [(301, 199), (210, 199), (321, 199)]
	mouse5_right: 3 runs with shapes: [(341, 199), (212, 199), (191, 199)]
	mouse6_left: 4 runs with shapes: [(302, 199), (157

In [4]:
# Print the features count
for datagroup, mice in data.items():
    for mouse_direction, runs in mice.items():
        for run, df in runs.items():
            print(f"Number of features: {df.shape[1] + 1 - 4}") # +1 bc of index, -4 to exclude frame, forestep, hindstep, and time
            break
        break
    break

Number of features: 196


In [5]:
def segment_steps_by_phase(df, phase_col="phase"):
    phases = df[phase_col]

    # 1. Find where phase changes
    changes = phases != phases.shift()
    change_points = df.index[changes]

    # 2. Get the phases at these change points
    change_phases = phases.loc[change_points].reset_index(drop=True)

    # 3. Identify "stance" → ... → "swing" → "stance" sequences
    segments = []
    i = 0
    while i < len(change_phases) - 2:
        if change_phases[i] == "stance":
            swing_found = False
            for j in range(i+1, len(change_phases)):
                if change_phases[j] == "swing":
                    swing_found = True
                elif swing_found and change_phases[j] == "stance":
                    start_idx = change_points[i]
                    end_idx = change_points[j] - 1
                    segment = df.loc[start_idx:end_idx].drop(columns=["Step Phase Forelimb", "Step Phase Hindlimb"], errors='ignore')
                    segments.append(segment)
                    i = j - 1  # skip ahead
                    break
            else:
                # no closing stance; go to end
                start_idx = change_points[i]
                segment = df.loc[start_idx:].drop(columns=["Step Phase Forelimb", "Step Phase Hindlimb"], errors='ignore')
                segments.append(segment)
                break
        i += 1

    return segments

segmented_hindsteps = []
segmented_foresteps = []
for group in data:
    for mouse in data[group]:
        for run in data[group][mouse]:
            df = data[group][mouse][run]
            to_drop = ['Frame', "Time (s)"]
            df = df.drop(columns=to_drop, errors='ignore')
            hindsteps = segment_steps_by_phase(df, phase_col="Step Phase Hindlimb")  # or the correct column name
            for step_df in hindsteps:
                segmented_hindsteps.append({
                    "step": step_df,
                    "group": group,
                    "mouse": mouse,
                    "run": run
                })
            foresteps = segment_steps_by_phase(df, phase_col="Step Phase Forelimb")  # or the correct column name
            for step_df in foresteps:
                segmented_foresteps.append({
                    "step": step_df,
                    "group": group,
                    "mouse": mouse,
                    "run": run
                })

In [6]:
# Flatten all steps into a single array to compute global mean/std
all_healthy_arrays = [step_dict["step"].values for step_dict in segmented_hindsteps if "pre" in step_dict["group"] and "left" in step_dict["mouse"]]
flat_data = np.vstack(all_healthy_arrays)

scaler = StandardScaler()
scaler = scaler.fit(flat_data)

In [7]:
def steps_to_tensor(step_dicts, scaler):
    """
        Convert a list of step dictionaries to a padded tensor of shape (num_steps, max_length, num_features).

        Returns:
            - A tensor of shape (num_steps, max_length, num_features) containing the scaled step data (B, T, F).
            - A tensor of lengths for each step indicating the actual length of each step (B).
    """
    step_arrays = [scaler.transform(sd["step"].values) for sd in step_dicts]
    lengths = [len(step) for step in step_arrays]
    max_len = max(lengths)
    dim = step_arrays[0].shape[1]

    padded = np.zeros((len(step_arrays), max_len, dim), dtype=np.float32)
    for i, arr in enumerate(step_arrays):
        padded[i, :len(arr)] = arr

    return torch.tensor(padded), torch.tensor(lengths)

healthy_steps = [s for s in segmented_hindsteps if "pre" in s["group"] and "left" in s["mouse"]]
step_tensor, lengths = steps_to_tensor(healthy_steps, scaler)

print(f"Step tensor shape: {step_tensor.shape}, \nLengths shape: {lengths.shape}")

Step tensor shape: torch.Size([560, 189, 196]), 
Lengths shape: torch.Size([560])


In [10]:
from src.LSTM import LSTMAutoencoder

def masked_mse_loss(pred, target, lengths):
    """
        Compute the masked mean squared error loss between predicted and target tensors.
    """
    mask = torch.arange(target.size(1))[None, :].to(lengths.device) < lengths[:, None]
    mask = mask.unsqueeze(-1).expand_as(target)  # (B, T, F)
    mse = (pred - target) ** 2
    masked_mse = (mse * mask).sum() / mask.sum()
    return masked_mse

class EarlyStopping:
    def __init__(self, patience=10, min_delta=1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.should_stop = False

    def step(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True

## Hyperparameters and early stopping
INPUT_DIM = step_tensor.shape[2]
HIDDEN_DIM = 64
LATENT_DIM = 16
BATCH_SIZE = 32
NUM_EPOCHS = 500
LR = 1e-3
PATIENCE = 50 # number of epochs to wait for improvement before stopping
MIN_DELTA = 1e-4 # minimum change to qualify as an improvement

# Data loader
train_idx, val_idx = train_test_split(np.arange(len(step_tensor)), test_size=0.2, random_state=42)
train_data = TensorDataset(step_tensor[train_idx], lengths[train_idx])
val_data = TensorDataset(step_tensor[val_idx], lengths[val_idx])

def collate_fn(batch):
    """
        Custom collate function to handle variable-length sequences.
        Returns a padded tensor and lengths.
    """
    x, lengths = zip(*batch)
    # print(f"element shapes: {[xi.shape for xi in x]}, lengths: {lengths}")
    return torch.stack(x).to(device), torch.stack(lengths).to(device)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, collate_fn=collate_fn)

# Model, optimizer
model = LSTMAutoencoder(INPUT_DIM, HIDDEN_DIM, LATENT_DIM)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

early_stopper = EarlyStopping(patience=PATIENCE, min_delta=MIN_DELTA)
best_val_loss = float('inf')
now = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filname = f'lstm_autoencoder_{now}.pt'
best_model_path = os.path.join(MODELS_DIR, f'lstm_autoencoder_{now}.pt')

# Training loop
for epoch in tqdm(range(NUM_EPOCHS)):
    t1 = timeit.default_timer()
    model.train()
    train_loss = 0
    for batch_x, batch_lens in train_loader:
        # batch_x, batch_lens = batch_x.to(device), batch_lens.to(device)
        # tqdm.write(f"Batch shape: {batch_x.shape}, Lengths shape: {batch_lens.shape}")
        # tqdm.write(f"Device of batch_x: {batch_x.device}, batch_lens: {batch_lens.device}")
        optimizer.zero_grad()
        pred, _ = model(batch_x, batch_lens)
        loss = masked_mse_loss(pred, batch_x, batch_lens)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_x, batch_lens in val_loader:
            # batch_x, batch_lens = batch_x.to(device), batch_lens.to(device)
            pred, _ = model(batch_x, batch_lens)
            val_loss += masked_mse_loss(pred, batch_x, batch_lens).item()

    avg_train = train_loss / len(train_loader)
    avg_val = val_loss / len(val_loader)
    t2 = timeit.default_timer()

    if epoch % 10 == 0 or epoch == NUM_EPOCHS - 1:
        tqdm.write(f"Epoch {epoch+1}, Train: {avg_train:.4f} - Val: {avg_val:.4f} - Time: {t2-t1:.2f}s")

    # Save model if validation loss improves
    if avg_val < best_val_loss:
        best_val_loss = avg_val
        best_state = model.state_dict()

    early_stopper.step(avg_val)
    if early_stopper.should_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

torch.save({'model_state_dict':best_state,
            'epoch': epoch,
            'val_loss': best_val_loss},
            best_model_path)
print(f"Best model saved at: {best_model_path} with val loss: {best_val_loss:.4f}")

Using device: cuda
LSTM device: cuda


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1, Train: 1.0146 - Val: 0.8357 - Time: 0.50s
Epoch 2, Train: 0.9717 - Val: 0.8092 - Time: 0.40s
Epoch 3, Train: 0.9432 - Val: 0.7834 - Time: 0.37s
Epoch 4, Train: 0.9279 - Val: 0.7687 - Time: 0.39s
Epoch 5, Train: 0.9102 - Val: 0.7596 - Time: 0.38s
Epoch 6, Train: 0.9025 - Val: 0.7519 - Time: 0.40s
Epoch 7, Train: 0.9025 - Val: 0.7482 - Time: 0.36s
Epoch 8, Train: 0.8896 - Val: 0.7427 - Time: 0.36s
Epoch 9, Train: 0.8821 - Val: 0.7387 - Time: 0.37s
Epoch 10, Train: 0.8709 - Val: 0.7363 - Time: 0.37s
Epoch 11, Train: 0.8715 - Val: 0.7310 - Time: 0.36s
Epoch 12, Train: 0.8699 - Val: 0.7334 - Time: 0.37s
Epoch 13, Train: 0.8673 - Val: 0.7325 - Time: 0.37s
Epoch 14, Train: 0.8519 - Val: 0.7242 - Time: 0.39s
Epoch 15, Train: 0.8520 - Val: 0.7200 - Time: 0.39s
Epoch 16, Train: 0.8492 - Val: 0.7207 - Time: 0.41s
Epoch 17, Train: 0.8488 - Val: 0.7196 - Time: 0.43s
Epoch 18, Train: 0.8383 - Val: 0.7201 - Time: 0.38s
Epoch 19, Train: 0.8501 - Val: 0.7391 - Time: 0.37s
Epoch 20, Train: 0.86

KeyboardInterrupt: 

In [ ]:
selected_steps = [
    s for s in segmented_hindsteps 
    if "left" in s["mouse"] 
]

step_tensor_all, lengths_all = steps_to_tensor(selected_steps, scaler)

## Get all embeddings for selected steps
model.eval()
with torch.no_grad():
    _, embeddings_all = model(step_tensor_all, lengths_all) # Shape: (B, T, F) -> (B, L)

In [ ]:
### Visualization of embeddings
umap_coords = UMAP(n_components=3, random_state=42).fit_transform(embeddings_all)

umap_df = pd.DataFrame({
    "UMAP1": umap_coords[:, 0],
    "UMAP2": umap_coords[:, 1],
    "UMAP3": umap_coords[:, 2],
    "Group": ["healthy" if "pre" in s["group"] else "unhealthy" for s in selected_steps],
    "Mouse": [s["mouse"] for s in selected_steps],
    "Run": [s["run"] for s in selected_steps],
    "Dataset": [s["group"] for s in selected_steps],
})

/home/filthyweeb/Documents/SeriousStuff/Work/MScThesis/.venv/lib64/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/filthyweeb/Documents/SeriousStuff/Work/MScThesis/.venv/lib64/python3.13/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [ ]:
def masked_mse_loss(pred, target, lengths):
    """
        Compute the masked mean squared error loss between predicted and target tensors.
    """
    mask = torch.arange(target.size(1))[None, :].to(lengths.device) < lengths[:, None]
    mask = mask.unsqueeze(-1).expand_as(target)  # (B, T, F)
    mse = (pred - target) ** 2
    masked_mse = (mse * mask).sum() / mask.sum()
    return masked_mse

class EarlyStopping:
    def __init__(self, patience=10, min_delta=1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.should_stop = False

    def step(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True

## Hyperparameters and early stopping
INPUT_DIM = step_tensor.shape[2]
HIDDEN_DIM = 64
LATENT_DIM = 16
BATCH_SIZE = 32
NUM_EPOCHS = 50
LR = 1e-3
PATIENCE = 10 # number of epochs to wait for improvement before stopping
MIN_DELTA = 1e-4 # minimum change to qualify as an improvement

# Data loader
train_idx, val_idx = train_test_split(np.arange(len(step_tensor)), test_size=0.2, random_state=42)
train_data = TensorDataset(step_tensor[train_idx], lengths[train_idx])
val_data = TensorDataset(step_tensor[val_idx], lengths[val_idx])
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

# Model, optimizer
model = LSTMAutoencoder(INPUT_DIM, HIDDEN_DIM, LATENT_DIM)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

early_stopper = EarlyStopping(patience=PATIENCE, min_delta=MIN_DELTA)
best_val_loss = float('inf')
now = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filname = f'lstm_autoencoder_{now}.pt'
best_model_path = os.path.join(MODELS_DIR, f'lstm_autoencoder_{now}.pt')

# Training loop
for epoch in range(NUM_EPOCHS):
    t1 = timeit.default_timer()
    model.train()
    train_loss = 0
    for batch_x, batch_lens in train_loader:
        optimizer.zero_grad()
        pred, _ = model(batch_x, batch_lens)
        loss = masked_mse_loss(pred, batch_x, batch_lens)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_x, batch_lens in val_loader:
            pred, _ = model(batch_x, batch_lens)
            val_loss += masked_mse_loss(pred, batch_x, batch_lens).item()

    avg_train = train_loss / len(train_loader)
    avg_val = val_loss / len(val_loader)
    t2 = timeit.default_timer()
    print(f"Epoch {epoch+1}, Train: {avg_train:.4f} - Val: {avg_val:.4f} - Time: {t2-t1:.2f}s")

    # Save model if validation loss improves
    if avg_val < best_val_loss:
        best_val_loss = avg_val
        best_state = model.state_dict()

    early_stopper.step(avg_val)
    if early_stopper.should_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

torch.save({'model_state_dict':best_state,
            'epoch': epoch,
            'val_loss': best_val_loss},
            best_model_path)
print(f"Best model saved at: {best_model_path} with val loss: {best_val_loss:.4f}")

Epoch 1, Train: 1.0072 - Val: 0.8566 - Time: 5.03s
Epoch 2, Train: 0.9664 - Val: 0.8274 - Time: 4.42s
Epoch 3, Train: 0.9419 - Val: 0.8054 - Time: 3.60s
Epoch 4, Train: 0.9233 - Val: 0.7907 - Time: 4.13s


KeyboardInterrupt: 

In [23]:
SCATTER_SIZE = 6
SCATTER_LINE_WIDTH = 1
SCATTER_SYMBOL = 'circle'
LEGEND_FONT_SIZE = 18
TITLE_FONT_SIZE = 24
AXIS_FONT_SIZE = 16
AXIS_TITLE_FONT_SIZE = 20

fig = px.scatter_3d(
    umap_df,
    x="UMAP1", y="UMAP2", z="UMAP3",
    color="Dataset",#"Group",                
    hover_data=["Mouse", "Run", "Dataset"],
    title="3D UMAP Projection of Step Embeddings"
)

fig.update_traces(marker=dict(size=SCATTER_SIZE, line=dict(width=SCATTER_LINE_WIDTH, color='DarkSlateGrey')))
fig.update_layout(
    legend=dict(title="Group", font=dict(size=LEGEND_FONT_SIZE)), template="plotly_white",
    width=900,
    height=700,
    scene=dict(
            xaxis=dict(title_font=dict(size=AXIS_TITLE_FONT_SIZE), tickfont=dict(size=AXIS_FONT_SIZE)),
            yaxis=dict(title_font=dict(size=AXIS_TITLE_FONT_SIZE), tickfont=dict(size=AXIS_FONT_SIZE)),
            zaxis=dict(title_font=dict(size=AXIS_TITLE_FONT_SIZE), tickfont=dict(size=AXIS_FONT_SIZE)),
        ),
    title=dict(font=dict(size=TITLE_FONT_SIZE))
    )
fig.show()